In [7]:
tcf(filename::String) = (@__DIR__) * "\\" * filename;

# Задача теплопроводности

Во многих инженерных задачах важным аспектом явлется знание распределения температуры в теле. 
Всем известно, что при нагреве, тела расширяются, что приводит к возникновению тепловых напряжений. Если известно количество подводимого и теряемого телом тепла, можно определенить распределение температуры в теле, а следовательно - распределение напряжений, знание которых важно при вращающихся механизмов, таких как реактивные двигатели, паровые генераторы и даже червячные редукторы.

## Уравнение переноса тепла

Стаицонарное уравнение теплопроводности в сплошной среде имеет вид 

$$
% \label{eq_eigen} \tag{1}
-\mathbf{k} \cdot \Delta T(\mathbf{x}) + q(\mathbf{x}) = 0, \text{ для } \mathbf{x}\in \Omega
$$

где $\mathbf{x} = \{x \hspace{0.1cm} y \hspace{0.1cm} z \}^T$ - вектор пространственных координат, $ \Omega $ - область решения задачи, определяема геометрией рассматриваемого тела,  $\mathbf{k} = \{k_{xx} \hspace{0.1cm} k_{yy} \hspace{0.1cm} k_{zz} \}^T$ - вектор коэффициентов теплопроводности в направлениях $ x $, $ y $ и $ z $, размерности $кВт / K$, и $q$ - внутренний источник тепла, размерности $кВт / м^3$. В уравнении неизвестным и является скалярное поле температур $T(\mathbf{x})$ и именно его неободимо определить при решении задачи.

Кроме определяющего уравнения обычно присутствуют дополнительные ограничения, задающие температуру или тепловой поток на поверхности тела, являющиеся граничными условиями для нашей задачи. Первый тип ограничений называют граничными условиями I-го рода, или условиям Дерихле

$$
    T(\mathbf{x}) = T_{D}(\mathbf{x}),\text{ для } \mathbf{x} \in {\partial {\Omega}_D}
$$

где $ \partial {\Omega}_D $ - граница области, на которой заданы условия Дерихле, а $T_D$ - температура на данной границе.

Второй тип ограничений называют граничными условиями II-го рода, или условиями Неймана

$$
    \nabla T(\mathbf{x}) \cdot \mathbf{n}(\mathbf{x}) = Q(\mathbf{x}), \text{ для } \mathbf{x}\in{\partial {\Omega}_N}
$$
где $ \partial {\Omega}_N $ - граница области, на которой заданы условия Неймана, $\mathbf{n}(\mathbf{x})$ - вектор нормали к границе, а $Q(\mathbf{x})$ - тепловой поток на данной границе.

## Формулировка задачи

В данной задаче необходимо рассчитать температурное поле радиатора охлаждения, подключенного к 

Данная задача будет рассмотрена в двухмерной постановке.

## Создание геометрии


Изначально родным для Gridap форматом геометрии является формат .json, имеющий определенную стркутуру. Однако более удобно использовать существующую библотеку gmsh для построения геометрии, имеющую большое количество возможностей и являющейся, по мнению авторов, наиболее мощной open-source библиотекой для построения расчетных неструктурированых сеток.


In [42]:
using Gmsh


gmsh.initialize()

gmsh.option.setNumber("General.Terminal", 0);
ms_max = 1;
ms_min = 0.1;
MSFC = 3;
EL_ORDER = 1
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", MSFC);
# gmsh.option.setNumber("Mesh.MeshSizeMax", diam/2);
# gmsh.option.setNumber("Mesh.MeshSizeMin", diam/5);
gmsh.option.setNumber("Mesh.MaxNumThreads3D", 8);
gmsh.option.setNumber("Mesh.ElementOrder", EL_ORDER);
gmsh.option.setNumber("Mesh.Algorithm3D", 9);
gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 2);

gmsh.model.add("t4")

height = 5;
width = 10;
rib_th = 1;
rib_num = 3;

sp = (width - rib_num * rib_th) / (rib_num - 1);

Lc1 = 0.3;

factory = gmsh.model.geo;
factory.addPoint(-width/2, 0,       0, Lc1, 1);
factory.addPoint( width/2, 0,       0, Lc1, 2);
factory.addPoint( width/2, height,  0, Lc1, 3);
factory.addPoint(-width/2, height,  0, Lc1, 4);

x1 = -width/2 + rib_th;
x2 = -width/2 + rib_th + sp;

factory.addPoint(x1, height, 0, Lc1, 5);
factory.addPoint(x1, rib_th, 0, Lc1, 6);
factory.addPoint(x2, rib_th, 0, Lc1, 7);
factory.addPoint(x2, height, 0, Lc1, 8);

x1 = -width/2 + 2rib_th + sp;
x2 = -width/2 + 2rib_th + 2sp;

factory.addPoint(x1, height, 0, Lc1, 9);
factory.addPoint(x1, rib_th, 0, Lc1, 10);
factory.addPoint(x2, rib_th, 0, Lc1, 11);
factory.addPoint(x2, height, 0, Lc1, 12);


# for i in 1:(rib_num-1)
#     p1x = -width/2 + rib_th + (sp + rib_th) * (i - 1);
#     p2x = p1x;
#     p3x = p1x + sp;
#     p4x = p3x;

#     factory.addPoint(p1x, height)
# end

factory.addLine(1, 2, 1);
factory.addLine(2, 3, 2);
factory.addLine(1, 4, 3);

for i in 4:11
    factory.addLine(i, i+1, i);
end
factory.addLine(12, 3, 12);


factory.addPhysicalGroup(1, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], -1, "free");
factory.addPhysicalGroup(1, [1], -1, "fixed");

factory.synchronize();

factory.addCurveLoop([-2, -1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 17);



factory.addPlaneSurface([17], 18)


factory.synchronize()

factory.addPhysicalGroup(2, [18], -1, "coller");

factory.synchronize()

gmsh.model.mesh.generate(2)


name = "geo.msh" |> tcf;
print(name)
gmsh.write(name)

gmsh.finalize()


c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_1\geo.msh

Полученная геометрия имеет две повехности - нижняя грань, по которой предполагается контакт с по верхностью, помечена меткой `"fixed"`, остальная часть границы помечена меткой `"free"`

# Загрузка геометрии в Gridap

Для загрузки .msh геометрии используется библиотека GridapGmsh, которая конвертирует в формат Gridap

In [43]:
using Gridap
using GridapGmsh

model = GmshDiscreteModel(name);



Info    : Reading 'c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_1\geo.msh'...
Info    : 25 entities
Info    : 400 nodes
Info    : 798 elements
Info    : Done reading 'c:\Users\kutsj\OneDrive\Education\Programming\Julia\CCM_tut\lesson_1\geo.msh'


UnstructuredDiscreteModel()

Для просмотра геометрии удобнее всего использовать Paraview:

In [44]:
vtk_file = "model" |> tcf;
writevtk(model, vtk_file);

В результате появится 3 файла - `"model_0.vtu"`, `"model_1.vtu"` и `"model_2.vtu"`, содержащие вершины, ребра и грани. 
Можно легко проверить границы которые имеются в импортированном файле.

![image](free.png)
![image](fixed.png)

# Конечнолементное пространство

Как только мы получим дискретизацию рассчетной области, следующим шагом будет создание дискретной аппроксимации пространств конечных элементов $V_0$ и $U_g$ (т.е. КЭ-пространств тестовых и пробных функций) задачи. Для этого, во-первых, мы собираемся построить дискретизацию $V_0$ в качестве стандартного соответствующего лагранжева КЭ-пространства (с нулевыми граничными условиями), связанного с дискретизацией вычислительной области. Аппроксимация пространства FE V_0 строится следующим образом:

In [45]:
reffe = ReferenceFE(lagrangian,Float64,EL_ORDER)
V0 = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags="fixed");

Здесь мы использовали конструктор `TestFESpace`, который создает конкретное КЭ-пространство (для использования в качестве тестового пространства) из набора параметров, описанных как позиционные и именнованные аргументы. Первый позиционный аргумент - это модель, поверх которой мы хотим построить пространство. Второй позиционный аргумент содержит информацию о типе интерполяции FE (в данном случае эталонный FE). С помощью ReferenceFE(lagrangian,Float64,EL_ORDER) мы выбираем скалярно-значный лагранжианский эталонный FE порядка 1, где значение функций формы будет представлено 64-разрядными числами с плавающей запятой. С помощью аргумента соответствия ключевому слову мы определяем регулярность интерполяции на границах ячеек сетки. Здесь мы используем соответствие =:H1, что означает, что результирующее интерполяционное пространство является подмножеством $H^1 (\Omega)$ (т.е. непрерывных функций формы). С другой стороны, мы передаем идентификаторы границы Дирихле через аргумент `dirichlet_tags`. В этом случае мы помечаем как Dirichlet все объекты дискретной модели, идентифицированные тегом `"fixed"`. Поскольку это тестовое пространство, соответствующие функции формы обращаются в нуль на границе Дирихле.

Как только пространство $V_0$ дискретизировано, мы приступаем к аппроксимации пробного пространства $U_g$.

In [54]:
g(x) = 60.0
Ug = TrialFESpace(V0,g)

TrialFESpace()

С этой целью мы использовали конструкторы `TrialFESpace`. Обратите внимание, что при построении пробного пространства мы передали функцию, представляющую значение граничного условия Дирихле.

## Численное интегрирование



In [55]:
degree = EL_ORDER*2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

GenericMeasure()

In [56]:
neumanntags = ["free"]
Γ = BoundaryTriangulation(model,tags=neumanntags)
dΓ = Measure(Γ,degree)

GenericMeasure()

In [77]:
h(x) = 0.0
a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ + ∫(0.1*u*v)*dΓ
b(v) = ∫( v*h )*dΓ

b (generic function with 1 method)

In [78]:
op = AffineFEOperator(a,b,Ug,V0)

AffineFEOperator()

In [79]:
ls = LUSolver()
solver = LinearFESolver(ls)

LinearFESolver()

In [80]:
uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 638
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 1995080348484422239

In [81]:
results_file = "results" |> tcf;
writevtk(Ω,results_file,cellfields=["uh"=>uh])

(["c:\\Users\\kutsj\\OneDrive\\Education\\Programming\\Julia\\CCM_tut\\lesson_1\\results.vtu"],)